In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# CPU cluster name
amlcompute_cluster_name = "ccluster-exer01"

# requisite from the rubric: Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Cluster already exist.')
except ComputeTargetException:
    print('Creating requested computer cluster, please wait...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
#libraries required for the Param Sampling
from azureml.train.hyperdrive import choice, loguniform
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': uniform(0.05, 1),
        '--max_iter': choice(20, 40, 60, 80, 100)
    }
)

# Specify a Policy
### YOUR CODE HERE: Early termination policy ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
training_folder = "./training"
est = SKLearn(source_directory=training_folder, entry_script = "train.py", compute_target = aml_compute)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output = True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:', parameter_values[3])

#registering the model
best_model = best_run.register_model(model_name='bankmarketing-hyper_model', model_path='azureml-logs/bmhypermodel.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path = [
       'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
   ]
dsbmarketing = TabularDatasetFactory.from_delimited_files(path=web_path, separator=',')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
sys.path.append("training")
from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(dsbmarketing)
xds_train, xds_test, yds_train, yds_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
#automl_config = AutoMLConfig(
#    experiment_timeout_minutes=30,
#    task=,
#    primary_metric=,
#    training_data=,
#    label_column_name=,
#    n_cross_validations=)
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dsbmarketing,
    label_column_name='y',
    n_cross_validations=5,
    max_concurrent_iterations= 4,
    max_cores_per_iteration= -1,    
    iterations=50,    
    compute_target=aml_compute,
    enable_onnx_compatible_models=True
    )

automl_exp = Experiment(workspace=ws, name="udacity-automlproject")

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
aml_marketing_run = automl_exp.submit(automl_config, show_output = True)
RunDetails(aml_marketing_run).show()
aml_marketing_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_aml_marketing_model = aml_marketing_run.get_output()
from azureml.automl.runtime.onnx_convert import OnnxConverter
best_aml_marketing_model, onnx_model= aml_marketing_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")
print(best_aml_marketing_model)

In [ ]:
#once the experiments are completed, delete the cluster
aml_compute.delete()